# P5 - Convolutioneel neuraal netwerk

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential


2022-03-26 17:31:14.422987: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-26 17:31:14.423005: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Data preparation

In [2]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print("x_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("x_test shape", X_test.shape)
print("y_test shape", y_test.shape)

x_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
x_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


Voor one hot encoding uit over de target values, en laten we ook de dataset normaliseren

In [3]:
# One hot encodign over de targets heen
Y_train = utils.to_categorical(y_train, 10)
Y_test = utils.to_categorical(y_test, 10)

# normaliseren van de data
X_train = utils.normalize(X_train)
X_test = utils.normalize(X_test)

## Modelling

Voor het berekenen van de output dimension kunnen we de volgende formule gebruiken.

    Output height = (Input height + padding height top + padding height bottom - kernel height) / (stride height) + 1 
    
    Output width = (Output width + padding width right + padding width left - kernel width) / (stride width) + 1

Aangezien het voor dit dataset altijd uitkomt naar een vierkant hoeven we maar 1 te gebruiken om de size te weten.

In [4]:
cnn = Sequential()

# Layer 1 (CN)
# mijn hoop is door de size voor nu nog het zelfe de te houden dat we meer detail kunnen opvangen.
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", input_shape=(32,32,3)))
cnn.add(BatchNormalization(axis=-1))
cnn.add(Activation("relu"))


# Layer 2 (CN)
cnn.add(Conv2D(filters=32, kernel_size=(3, 3), strides=(2, 2)))
cnn.add(BatchNormalization(axis=-1))
cnn.add(Activation("relu"))

# Layer 3 (CN)
cnn.add(Conv2D(filters=16, kernel_size=(3, 3)))
cnn.add(BatchNormalization(axis=-1))
cnn.add(Activation("relu"))
cnn.add(MaxPool2D(pool_size=(2, 2)))
cnn.add(Flatten())

# Layer 4 (FC)
cnn.add(Dense(512))
cnn.add(BatchNormalization(axis=-1))
cnn.add(Activation("relu"))

# Layer 5 (FC)
cnn.add(Dense(200))
cnn.add(BatchNormalization(axis=-1))
cnn.add(Activation("relu"))

# Layer 6 (FC)
cnn.add(Dense(10))
cnn.add(Activation("softmax"))


2022-03-26 17:31:16.593780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-26 17:31:16.593803: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-26 17:31:16.593817: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Barry-EOS): /proc/driver/nvidia/version does not exist
2022-03-26 17:31:16.593998: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Summary
Hieronder kunnen we dan de summary zien van het gehele netwerk

In [5]:
cnn.summary(
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 15, 15, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_1 (Activation)   (None, 15, 15, 32)        0

### Compilen
Ook hier moeten we het model gaan compilen met een loss function en optimizer.

In [6]:
cnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Trainen en evalueren 
Hierna kunnen we het model gaan fitten. Dit gaan we op een beetje een apparte manier doen aangezioen we de data willen over lappen met de evalueren van de test set voor overfitting.

In [7]:
from datetime import datetime
# Define the Keras TensorBoard callback.
logdir="logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


In [8]:
cnn.fit(X_train, Y_train, batch_size=64, epochs=20, verbose=1, validation_data=(X_test, Y_test), callbacks=tensorboard_callback)

Epoch 1/20


2022-03-26 17:31:17.083572: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 614400000 exceeds 10% of free system memory.


782/782 [==============================] - 19s 23ms/step - loss: 1.5668 - accuracy: 0.4391 - val_loss: 1.8076 - val_accuracy: 0.3684
Epoch 2/20
782/782 [==============================] - 17s 22ms/step - loss: 1.2510 - accuracy: 0.5590 - val_loss: 1.8784 - val_accuracy: 0.3670
Epoch 3/20
782/782 [==============================] - 17s 22ms/step - loss: 1.0829 - accuracy: 0.6193 - val_loss: 1.8644 - val_accuracy: 0.3806
Epoch 4/20
782/782 [==============================] - 18s 22ms/step - loss: 0.9491 - accuracy: 0.6671 - val_loss: 1.5125 - val_accuracy: 0.5064
Epoch 5/20
782/782 [==============================] - 18s 23ms/step - loss: 0.8237 - accuracy: 0.7099 - val_loss: 1.7650 - val_accuracy: 0.4586
Epoch 6/20
782/782 [==============================] - 17s 22ms/step - loss: 0.7002 - accuracy: 0.7548 - val_loss: 1.6898 - val_accuracy: 0.4877
Epoch 7/20
782/782 [==============================] - 17s 21ms/step - loss: 0.5811 - accuracy: 0.7983 - val_loss: 2.3541 - val_accuracy: 0.4461
Epo

## Stap 4. Conclussie

Ook hier kunnen we kijken hoe goed de resultaten zijn.

De volgende data heb ik verzameld van de het trainen van het netwerk, voor het verzamelen van de data heb ik tensorboard gebruikt vandaar dat er ook wat extra code voor callbacks staan

<img src="Screenshot_20220326_174414.png">

Okay door het model te trainen en dan nog te evalueren kunnen we concludderen dat het model Zeer erg overfit is en geen goed model is geworden.\
Ik zou kunnen proberen om een ander architect te maken die beter werkt maar om eerlijk te zijn weet ik niet zo heel snel wat ik zou moeten veranderen om een beter werkend netwerk te krijgen.\
En ik moet ok nog de opdrachten van HPP maken.